In [1]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime, timedelta
from tqdm import tqdm
import sys

In [2]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makePageUrl():
    urls = []
    date = datetime.now() - timedelta(days=1)
    formated_date = date.strftime('%Y%m%d')

    for i in range(1,31):
        pg = str(i)
        url = "https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=" + formated_date+ "&page=" + pg
        urls.append(url)
    return urls    


In [3]:
page_urls = makePageUrl()

In [4]:
page_urls

['https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=2',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=3',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=4',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=5',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=6',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=7',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=8',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=9',
 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20230815&page=10',

In [5]:
#ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler_head(page_url,n): #n은 1~10
    #html 불러오기
    original_html = requests.get(page_url,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    url_news = html.select_one("#main_content > div.list_body.newsflash_body > ul.type06_headline > li:nth-child("+ str(n) +") > dl > dt:nth-child(2) > a")
    url = url_news.attrs['href']
    return url

def articles_crawler(page_url,n): #n은 1~10
    #html 불러오기
    original_html = requests.get(page_url,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_news = html.select_one("#main_content > div.list_body.newsflash_body > ul.type06 > li:nth-child("+ str(n) +") > dl > dt > a")
    url = url_news.attrs['href']
    return url

In [6]:
!pip freeze >> requirements.txt

In [7]:
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_press = []

for i in page_urls:
    for m in range(1,11):
        try:
            url = articles_crawler_head(i,m)
            news_url.append(url)
        except AttributeError:
            continue
        
        try:
            url = articles_crawler(i,m)
            news_url.append(url)
        except AttributeError:
            pass

In [8]:
news_urls = list(set(news_url))

In [9]:
print(len(news_urls))
news_urls


96


['https://n.news.naver.com/mnews/article/215/0001119344?sid=101',
 'https://n.news.naver.com/mnews/article/011/0004226535?sid=101',
 'https://n.news.naver.com/mnews/article/421/0006991065?sid=101',
 'https://n.news.naver.com/mnews/article/366/0000924508?sid=101',
 'https://n.news.naver.com/mnews/article/417/0000942136?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014131683?sid=101',
 'https://n.news.naver.com/mnews/article/014/0005057254?sid=101',
 'https://n.news.naver.com/mnews/article/417/0000942137?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014131456?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004879992?sid=101',
 'https://n.news.naver.com/mnews/article/029/0002819407?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005551696?sid=101',
 'https://n.news.naver.com/mnews/article/468/0000971008?sid=101',
 'https://n.news.naver.com/mnews/article/658/0000049558?sid=101',
 'https://n.news.naver.com/mnews/article/003/0012031803?sid=101',
 'https://

In [10]:
# 뉴스 내용 크롤링

for i in tqdm(news_urls):
    #각 기사 html get하기
    news = requests.get(i)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 언론사 가져오기
    try:
      press = news_html.select_one(".media_end_head_top_logo > img")['title']
    except TypeError:
      press = news_html.select_one("#_CHANNEL_LAYER_366_0000868613 > span.media_end_head_top_channel_layer_text > strong")
    
    # 언론사 코드 가져오기
    #press_code = final_urls[39:41]

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    press = re.sub(pattern=pattern1, repl='', string=str(press))
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)
    news_press.append(press)
    
    #날짜 가져오기
    
    try:
        html_date = news_html.select_one("._ARTICLE_DATE_TIME")
        news_date = html_date.get_text('data-date-time')
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        
        

    news_dates.append(news_date)

print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(news_urls)
print("\n[뉴스 내용]")
print(news_contents)
print("\n[언론사]")
print(news_press)

print('news_title: ',len(news_titles))
print('news_url: ',len(news_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))
print('news_press: ',len(news_press))

  0%|          | 0/96 [00:00<?, ?it/s]

100%|██████████| 96/96 [00:12<00:00,  7.47it/s]


[뉴스 제목]
["카카오·케이뱅크 '주담대' 덕분 실적…토스뱅크, 흑자 전환", '열기 꺾인 2차전지…보름새 시총 30조 증발', '원·달러 환율 상승세', '카카오·케이뱅크, 주담대 큰 폭 증가… 올 상반기에만 5.4조원 늘어', '씨티은행 사태 재현 없다… 다음달부터 은행업 일부 폐업도 인가 받아야', "미래산업 육성 '서울비전펀드' 올해 1조원 조성…4년간 총 5조원", '“대기업대출 30% 늘릴 동안 기술기업대출 10% 줄였다” 중소·벤처 외면하는 시중은행', 'SC제일·씨티은행, 이자이익만 1.1조 넘어… 고금리에 NIM 상승', '주담대 기대 성장한 카카오·케이뱅크…6개월 새 5.4조원↑', "외국계 은행 상반기 실적 '희비'", '"화물차 우회전 사고 막아라" 신한카드, 예방 캠페인 전개', '손보 빅5, 반기 만에 순익 4조 돌파…사별 희비 엇갈려', '금융지주·은행 CEO 보수 1위, 한국씨티은행장 18억6000만원 ‘지주 회장보다 많다’', '이차전지주 보름새 시총 30조 증발…고점 지났나', '카드론 금리 또 오를까', '올해 상반기 채무조정 신청자수 9만명↑, 빚 갚는 데 100.5개월', '금감원 고객만족도 첫 하락…민원·분쟁조정 만족도 60점대', '손보 `빅5`, 상반기 순이익 4조 돌파', '[톡톡금융인] "서류 없어도 보험금 지급…디지털 혁신으로 고객 편의 강화"', "올해 100조 이상 '한은 마통' 쓴 정부, 세수 부족에 유류세 인하 고심", '"금감원 민원·분쟁조정 답답해요" 만족도 61.6점, 1년새 10점 하락', '깊어지는 ‘빚의 늪’…한계 내몰린 채무조정 신청자 폭증', "'달러 강세'", '[인사이드 스토리]서류 부담 던 GA설계사, 생보 깨울까', '주담대 만 34세 이하로 제한?… "4050세대 소외" "가계대출 고삐" [어떻게 생각하십니까]', '환율 급등에도 달러예금 썰물…"지금이 고점" 판단 차익실현', '은행권 점포 폐쇄 계속되는데…5년간 지점 650곳 늘린 은행도 있다', '“가맹점 사용액 0.2~0.

In [11]:
news_contents

['[\n\n\n\n\n인터넷전문은행들이 올해 상반기 주택담보대출 성장에 힘입어 견조한 실적을 냈다.카카오뱅크의 상반기 순익은 1년 전보다 48% 늘어난 1천838억원으로 집계됐다.올해 상반기 충당금 1천117억원을 적립하고도 역대 최대실적을 기록했다.케이뱅크의 상반기 순익은 250억원으로 작년 상반기(457억원)보다 46% 줄었다.이자이익(2천97억원)과 비이자이익(155억원) 모두 1년 전보다 22%, 278% 증가했으나 대손충당금 전입액이 지난해 상반기(494억원)의 2배가 넘는 1천205억원으로 늘어나면서 순이익은 감소했다.카카오뱅크와 케이뱅크의 실적을 지탱한 것은 대출, 특히 폭발적으로 늘어난 주담대였다.카카오뱅크와 케이뱅크가 낮은 금리를 내세워 공격적인 영업을 이어가면서 올해 상반기 인터넷은행 두 곳에서만 주담대(전월세 대출 포함) 잔액이 5조4천360억원(34.9%) 늘었다.구체적으로 카카오뱅크의 주담대 잔액은 지난해 말 13조2천960억원에서 6월 말 17조3천220억원으로 4조260억원(30.3%) 증가했다.카카오뱅크는 지난 2월 주담대 \'갈아타기(대환)\' 고객을 대상으로 최대 0.6%p(포인트) 금리 할인을 제공하고, 3월에는 전월세보증금 대출 특판, 4월에도 주담대 특판을 진행한 바 있다.케이뱅크의 주담대 잔액 역시 같은 기간 2조2천930억원에서 3조7천억원으로 1조4천70억원(61.4%) 늘었다.케이뱅크도 지난해 10월부터 아파트담보대출 금리를 6차례 인하하며 올해 들어 7월 말까지 1조6천억원을 신규 취급했다.전체 여신 중 주택담보대출 비율은 카카오뱅크가 51.1%, 케이뱅크가 29.2%에 달했다. 지난해 말(47.7%, 21.3%)보다 그 비중이 확대됐다.토스뱅크는 올해 상반기 실적을 아직 공개하지 않았지만, 지난 7월 최초로 흑자전환에 성공했다고 밝혔다. 지난 2021년 10월 출범 후 22개월 만이다.7월 말 기준 흑자 규모는 약 10억원으로, 토스뱅크 관계자는 "큰 외부 변수가 없는 한 7월 흑자 전환이 3분기 흑자로 이어질

In [14]:

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'press':news_press,'title':news_titles,'link':news_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.now() 
news_df.to_csv('{}_{}.csv'.format('news',now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  96
